# Client Test

In [23]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [24]:
the_client = client.Client()
the_client.start()

My name is Client_366

[connected: ('192.168.0.100', 9662)]
Sending 305 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-01-31 19:19:35.580300'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-01-31 19:19:35.580300'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

[Listen to messages]

Data received: 399 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 19:19:35.580300'), ('message_id', '2017-01-31 19:19:35.656300'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_f1d30800': "('192.168.0.103', 4592)", 'NodeMCU_1dsc000': "('192.168.0.101', 1857)", 'Client_366': "('192.168.0.100', 55493)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-31 19:19:35.656300'), ('type', 'result')])



### Prepare messages

In [25]:
messages = OrderedDict()

messages['read_GPIOs'] = {'type': 'command',
                          'command': 'read GPIOs',
                          'kwargs': {'pins': (5, 12, 13, 14, 15, 16)},
                          'need_result': True}

messages['blink_led'] = {'type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 10, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}} 

In [26]:
# messages['write_GPIOs'] = {'type': 'command',
                           # 'command': 'write GPIOs',
                           # 'kwargs': {'pins_and_values': [(2, 0), (2, 1), (2, 0),]}} 

# messages['test eval'] = {'type': 'eval',
                         # 'to_evaluate': '2+3',
                         # 'need_result': True}                                   

# messages['test exec'] = {'type': 'exec',
                         # 'to_exec': 'print("Testing exec !")'}

# with open('script_to_deploy.py') as f:
    # script = f.read()        
# messages['test upload script'] = {'type': 'script', 
                                  # 'script': script} 

### Ask Hub for a list of connected nodes

In [27]:
while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')


Data received: 166 bytes
Message:
OrderedDict([('info', 'Just check to see if you are still there. No reply needed.'), ('receiver', 'all workers'), ('sender', 'Hub'), ('type', 'info')])



In [28]:
message = {'type': 'command',
                   'command': 'list connections by name',
                   'need_result': True}     

_, asynch_result = the_client.request('Hub', message) 

try:
    remote_nodes = sorted(list(asynch_result.get().keys()))

    print ('\n[____________ Connected nodes ____________]\n')        
    print('\nConnected nodes:\n{}\n'.format(remote_nodes)) 
    
except Exception as e:
    print(e)

Sending 276 bytes
Message:
OrderedDict([('command', 'list connections by name'), ('correlation_id', '2017-01-31 19:20:01.962300'), ('message_id', '2017-01-31 19:20:01.962300'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])


Data received: 399 bytes
Message:
OrderedDict([('correlation_id', '2017-01-31 19:20:01.962300'), ('message_id', '2017-01-31 19:20:02.050300'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'NodeMCU_f1d30800': "('192.168.0.103', 4592)", 'NodeMCU_1dsc000': "('192.168.0.101', 1857)", 'Client_366': "('192.168.0.100', 55493)"}), ('sender', 'Hub'), ('time_stamp', '2017-01-31 19:20:02.050300'), ('type', 'result')])


[____________ Connected nodes ____________]


Connected nodes:
['Client_366', 'NodeMCU_1dsc000', 'NodeMCU_f1d30800']



### Read one GPIO pin

In [30]:
for remote_node in remote_nodes:
    _, result = the_client.request(remote_node, messages['read_GPIOs'] ) 
    print('\nGPIO status for {}: {}\n'.format(remote_node, result.get()))

Sending 314 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-31 19:21:02.747300'), ('kwargs', {'pins': (5, 12, 13, 14, 15, 16)}), ('message_id', '2017-01-31 19:21:02.747300'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])


Data received: 358 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-31 19:21:02.747300'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2017-01-31 19:21:02.747300'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('time_stamp', '2017-01-31 19:21:02.806300'), ('type', 'command')])


Processed result:
OrderedDict([('correlation_id', '2017-01-31 19:21:02.747300'), ('message_id', '2017-01-31 19:21:02.874300'), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'result')])

Sending 222 bytes
Message:
OrderedDict

### Blink each node

In [31]:
# remote_node = 'NodeMCU_|edca00'
# the_client.request(remote_node, messages['blink_led']) 

for remote_node in remote_nodes:
    the_client.request(remote_node, messages['blink_led']) 

Sending 329 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-01-31 19:21:19.288300'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 10}), ('message_id', '2017-01-31 19:21:19.288300'), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

Sending 334 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-01-31 19:21:19.337300'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 10}), ('message_id', '2017-01-31 19:21:19.337300'), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])


Data received: 373 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-01-31 19:21:19.288300'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 10}), ('message_id', '2017-01-31 19:21:19.288300'), ('receiver', 'Client_366'), ('reply_to'

### Send out messages and get asynchonous results

In [32]:
print ('\n[______________ Sending messages ______________]\n')
                
results = []

## send out the messages
for message in messages.values():
    for remote_node in remote_nodes:
        if remote_node != the_client.node.worker.name:  # exclude client self
            formatted_message, asynch_result = the_client.request(remote_node, message) 
            results.append((formatted_message, asynch_result))


[______________ Sending messages ______________]

Sending 319 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-31 19:21:27.266300'), ('kwargs', {'pins': (5, 12, 13, 14, 15, 16)}), ('message_id', '2017-01-31 19:21:27.266300'), ('need_result', True), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

Sending 320 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-01-31 19:21:27.312300'), ('kwargs', {'pins': (5, 12, 13, 14, 15, 16)}), ('message_id', '2017-01-31 19:21:27.312300'), ('need_result', True), ('receiver', 'NodeMCU_f1d30800'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('type', 'command')])

Sending 334 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-01-31 19:21:27.346300'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 10}), ('message_id', '2017-01-31 19:21:27.346300'), ('receiver', 'No

### Actually get the results

In [33]:
# collect and print results        
print('\n[_________ Wait few seconds for reply _________]\n')

for (message, result) in results:
    try:
        if message.get('need_result'):
            print('\n[Result for request]:\n___Request___:\n{0}\n___Result____:\n{1}\n'.format(message, result.get() if result else None))
    except Exception as e:
        print('\n[{}]\nMessage:\n{}'.format(e, message))


[_________ Wait few seconds for reply _________]


[Result for request]:
___Request___:
{'sender': 'Client_366', 'kwargs': {'pins': (5, 12, 13, 14, 15, 16)}, 'correlation_id': '2017-01-31 19:21:27.266300', 'type': 'command', 'reply_to': 'Client_366', 'message_id': '2017-01-31 19:21:27.266300', 'need_result': True, 'receiver': 'NodeMCU_1dsc000', 'command': 'read GPIOs'}
___Result____:
[[5, 0], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]


[Result for request]:
___Request___:
{'sender': 'Client_366', 'kwargs': {'pins': (5, 12, 13, 14, 15, 16)}, 'correlation_id': '2017-01-31 19:21:27.312300', 'type': 'command', 'reply_to': 'Client_366', 'message_id': '2017-01-31 19:21:27.312300', 'need_result': True, 'receiver': 'NodeMCU_f1d30800', 'command': 'read GPIOs'}
___Result____:
[[5, 0], [12, 1], [13, 1], [14, 1], [15, 0], [16, 1]]



### Stop the demo

In [34]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')


[________________ Demo stopped ________________]

